In [1]:
import numpy as np

import os



In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

In [225]:
import hyper_pixelNN as hsinn

In [228]:
reload(hsinn)

<module 'hyper_pixelNN' from 'hyper_pixelNN.py'>

In [3]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipympl

In [283]:
def str_to_spatial_spectral_receptive_field_size(s):
    """
    Coming in in (b,h,w)
    """
    filters = s.split('_')
    filters = [f.split('-') for f in filters]
    filters = [[int(n) for n in f] for f in filters]
    psi1, psi2, phi = filters
    b1,h1,_ = psi1
    b2,h2,_ = psi2
    b3,h3,_ = phi
    # return (h1+h2+h3+2, b1+b2+b3+2)
    # multiply by 3 in spectral bc there is downsampling there
    return (h1+h2+h3-2, b1*3+b2*3+b3*3-2)

In [279]:
def results_to_grid(result_dict):

    winds = set()
    bands = set()
    best_acc = 0
    for k, v in result_dict.iteritems():
        w, b = str_to_spatial_spectral_receptive_field_size(k)
        winds.add(w)
        bands.add(b)
        best_acc = max(best_acc, sum(v) / float(len(v)))
    winds = sorted(list(winds))
    bands = sorted(list(bands))
    Z = np.zeros((len(winds), len(bands)))
#     Z[:,:] = np.nan
    
    for k, v in result_dict.iteritems():
        w, b = str_to_spatial_spectral_receptive_field_size(k)
        i = winds.index(w)
        j = bands.index(b)
        acc = sum(v) / float(len(v))
        if acc == best_acc:
            print('Best acc (%.1f) is for: %s' % (best_acc * 100, k) )
        if Z[i,j] != 0:
            print('Conflict: %s' % k)
        if Z[i,j] < acc:
            Z[i,j] = acc
        
    X, Y = np.meshgrid(np.array(bands),winds)
    
    return X, Y, Z

In [280]:
def tap2wavelength(M, dataset):
    """Converts tap lengths to nm for a dataset's spectral axis.
    """
    return M*hsinn.bandwidth_dict[dataset]

def tap2meters(M, dataset):
    """Converts tap lengths to meters for a dataset's spatial axis.
    """
    return M*hsinn.spatial_res_dict[dataset] 
    

In [295]:
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_KSC_singlesite_s03_10trials.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_Bots_singlesite_s03_10trials.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_s03_10trials_full.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_KSC_singlesite_s03_10trials_new.npz'
#npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_Bots_singlesite_s03_10trials_new.npz'
#npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_s50_10trials_full_new.npz'


In [207]:

npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_s200_3trials.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_s50_10trials.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_s03_10trials_full_new.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_paviaU_s03_10trials.npz'



In [296]:
results = np.load(npz_name)['results'].item()

In [162]:
# add to results
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_KSC_singlesite_s03_10trials_append9.npz'
npz_name = '/scratch0/ilya/locDoc/pyfst/gridsearch_Bots_singlesite_s03_10trials_append9.npz'

for k,v in np.load(npz_name)['results'].item().iteritems():
    #print([k,sum(v) / len(v)])
    results[k] = v

In [297]:
X, Y, Z = results_to_grid(results)
dataset = 'KSC'
#dataset = 'Botswana'

Conflict: 3-5-5_3-5-5_3-5-5
Conflict: 7-5-5_7-5-5_7-5-5
Best acc (71.2) is for: 5-9-9_5-7-7_5-7-7
Conflict: 9-5-5_9-5-5_9-5-5
Conflict: 5-9-9_5-3-3_5-3-3


In [298]:
results['5-9-9_5-7-7_5-7-7']

[0.7556071152358855,
 0.6376643464810519,
 0.7469064191802011,
 0.7248646558391338,
 0.7088167053364269,
 0.7378190255220418,
 0.7134570765661253,
 0.7146171693735499,
 0.7165506573859242,
 0.6653132250580046]

In [169]:
# fill Z with nan if no accuracy was recorded
for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        if Z[i,j] == 0:
            Z[i,j] = np.nan


In [172]:
# cut off the row with missing values
Z = Z[:,:3]
X = X[:,:3]
Y = Y[:,:3]

In [292]:
X = tap2wavelength(X, dataset)
Y = tap2meters(Y, dataset)

In [293]:
bi, bj = [Z.argmax() // Z.shape[1], Z.argmax() % Z.shape[1]]
print('Best receptive field was %.2f nm, %.2f m' % (X[bi,bj], Y[bi,bj]))

Best receptive field was 529.34 nm, 450.00 m


In [286]:
plt.close('all')

In [294]:
fig = plt.figure()
ax = fig.gca(projection='3d')

real_Z = [z for z in Z.flatten() if not np.isnan(z)]

# Plot the surface.
# surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False, vmin=min(real_Z), vmax=max(real_Z))

# Plot the wireframe
norm = plt.Normalize(Z.min(), Z.max())
colors = cm.coolwarm(norm(Z))
rcount, ccount, _ = colors.shape
surf = ax.plot_surface(X, Y, Z, rcount=rcount, ccount=ccount,
                       facecolors=colors, shade=False, linewidth=4)
surf.set_facecolor((0,0,0,0))

# Customize the z axis.

ax.set_zlim(min(real_Z), max(real_Z))
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
ax.set_xlabel('Spectral Receptive Field (nm)')
ax.set_ylabel('Spatial Receptive Field (m)')
ax.set_zlabel('Accuracy')

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

FigureCanvasNbAgg()

Text(0.5,0,u'Accuracy')